In [264]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
%matplotlib inline
pd.set_option('max_colwidth',1000)

In [281]:
df = pd.read_csv("file_to_use_comparison (1).csv")

In [282]:
df['comment_body'].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             NaN
1                                                                                     

In [283]:
# remove first line: bot generated
df = df.iloc[1:].reset_index(drop=True)

In [284]:
#drop missing values
df = df.dropna(subset=["comment_body"]).reset_index(drop=True)

In [285]:
# clean comments
def clean_comment(comment):
    # Remove newline characters and tabs
    comment = re.sub(r'[\n\t]', ' ', comment)
    # Remove URLs (http, https, www)
    comment = re.sub(r'http\S+|www\S+', '', comment)
    # Remove any unwanted characters (optional, e.g., keeping only alphanumeric and punctuation)
    comment = re.sub(r'[^\w\s,.!?\'\"`-]', '', comment)
    # Remove extra spaces
    comment = re.sub(r'\s+', ' ', comment).strip()
    return comment
df['cleaned_comment'] = df['comment_body'].apply(clean_comment)

In [286]:
# create tags is_NASA, is_ISRO, is_Both, is_None
# Rules to create tags
## If comment contains ISRO and NASA, then is_Both = 1
## else If comment contains only ISRO, then is_ISRO = 1
## else If comment contains only NASA, then is_NASA = 1
## else If post_title contains ISRO and NASA, then is_Both = 1
## else If post_title contains only ISRO, then is_ISRO = 1
## else If post_title contains only NASA, then is_NASA = 1
## else is_None = 1
def create_tags_with_list(comment, post_title):
    # Initialize the tags
    tags = {
        'is_NASA': 0,
        'is_ISRO': 0,
        'is_Both': 0,
        'is_None': 0,
        'actual_comment': comment,
        'actual_title':post_title,
        'final_comment':"",
        'final_title':""
    }
    nasa_list = ["america", "nasa" ]
    isro_list = ["chandrayaan", "india", "gaganyaan", "mangalyaan", "pragyan", "isro"]
    # Convert comment and post_title to lowercase for case-insensitive comparison
    comment = comment.lower()
    post_title = post_title.lower()

    comment_words = comment.split()
    post_title_words = post_title.split()

    is_nasa_in_comment = any(x in nasa_list for x in comment_words)
    is_isro_in_comment = any(x in isro_list for x in comment_words)

    is_nasa_in_title = any(x in nasa_list for x in post_title_words)
    is_isro_in_title = any(x in isro_list for x in post_title_words)
    
    # Check the conditions based on the rules
    if is_nasa_in_comment and is_isro_in_comment:
        tags['is_Both'] = 1
    elif is_isro_in_comment:
        tags['is_ISRO'] = 1
    elif is_nasa_in_comment:
        tags['is_NASA'] = 1
    elif is_nasa_in_title and is_isro_in_title:
        tags['is_Both'] = 1
    elif is_isro_in_title:
        tags['is_ISRO'] = 1
    elif is_nasa_in_title:
        tags['is_NASA'] = 1
    else:
        tags['is_None'] = 1
    tags['final_comment'] = comment_words
    tags['final_title'] = post_title_words
     
    return tags


In [287]:
df_tags = df.apply(lambda row: create_tags_with_list(row['comment_body'], row['post_title']), axis=1)
# Convert the result of apply to a DataFrame (avoid list conversion which can reorder rows)
df_tags = pd.DataFrame(df_tags.tolist())
# Reset index on both DataFrames to ensure alignment
df = df.reset_index(drop=True)
df_tags = df_tags.reset_index(drop=True)
# Concatenate the original DataFrame with the tags
df_combined = pd.concat([df, df_tags], axis=1)#df = pd.concat([df, df_tags], axis=1)

In [288]:
print(df_combined['is_Both'].value_counts())
print(df_combined['is_NASA'].value_counts())
print(df_combined['is_ISRO'].value_counts())
print(df_combined['is_None'].value_counts())

is_Both
0    3012
1     246
Name: count, dtype: int64
is_NASA
0    2752
1     506
Name: count, dtype: int64
is_ISRO
0    1642
1    1616
Name: count, dtype: int64
is_None
0    2368
1     890
Name: count, dtype: int64
